In [35]:
#importamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest
from itertools import combinations


In [36]:
#defino función genérica para cargar CSV
def load_csv(path, nrows=None):
    df = pd.read_csv(path, nrows=nrows)
    return df

In [37]:
# Cargo manualmente los CSV de 2014 a 2023
df_2015 = load_csv("2015.csv")
df_2016 = load_csv("2016.csv")
df_2017 = load_csv("2017.csv")
df_2018 = load_csv("2018.csv")
df_2019 = load_csv("2019.csv")
df_2020 = load_csv("2020.csv")
df_2021 = load_csv("2021.csv")
df_2022 = load_csv("2022.csv")
df_2023 = load_csv("2023.csv")
df_2024 = load_csv("2024.csv")

# Verifico columnas de cada dataset
print("2015:", df_2015.columns)
print("2016:", df_2016.columns)
print("2017:", df_2017.columns)
print("2018:", df_2018.columns)
print("2019:", df_2019.columns)
print("2020:", df_2020.columns)
print("2021:", df_2021.columns)
print("2022:", df_2022.columns)
print("2023:", df_2023.columns)
print("2024:", df_2024.columns)


2015: Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual'],
      dtype='object')
2016: Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Lower Confidence Interval', 'Upper Confidence Interval',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual'],
      dtype='object')
2017: Index(['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high',
       'Whisker.low', 'Economy..GDP.per.Capita.', 'Family',
       'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual'],
      dtype='object')
2018: Index(['Overall rank', 'Country or region', 'Score', 'GDP per capita',
       'Social support', 'Healthy life 

In [38]:
df_2015 = pd.read_csv("2015.csv")


In [39]:
df_2015.shape


(158, 12)

In [40]:
df_2015.columns


Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual'],
      dtype='object')

In [41]:
df_2015.head()


,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [42]:
df_2015.dtypes


Country                           object
Region                            object
Happiness Rank                     int64
Happiness Score                  float64
Standard Error                   float64
Economy (GDP per Capita)         float64
Family                           float64
Health (Life Expectancy)         float64
Freedom                          float64
Trust (Government Corruption)    float64
Generosity                       float64
Dystopia Residual                float64
dtype: object

In [43]:
df_2015.isna().sum()


Country                          0
Region                           0
Happiness Rank                   0
Happiness Score                  0
Standard Error                   0
Economy (GDP per Capita)         0
Family                           0
Health (Life Expectancy)         0
Freedom                          0
Trust (Government Corruption)    0
Generosity                       0
Dystopia Residual                0
dtype: int64

In [44]:
# lista de años
years = list(range(2015, 2025))
dfs = []

for y in years:
    df = pd.read_csv(f"{y}.csv")
    df["year"] = y
    dfs.append(df)

# unir todos
df_all = pd.concat(dfs, ignore_index=True)


In [45]:
def pre_eda(df: pd.DataFrame, name: str = "df"): #una funcion pre-Eda que recibe un dataframe y lo llama df.
    rep = {} #define un diccionario donde iran guardando los resultados del preEda.


    dfc = df.copy()# se crea una copia del dataframe para no trabajar sobre el original.

    # Normaliza columnas
    rep["cols_original"] = list(dfc.columns)#Guarda el nombre de las columnas.Normaliza el texto.Convierte a minuscula,quita espacios,
                                            #cambia los espacios por guiones bajos y eliminia cualquier caracter raro con regex)
    dfc.columns = (
        dfc.columns.str.strip().str.lower()
        .str.replace(" ", "_", regex=True)
        .str.replace("[^0-9a-z_]", "", regex=True)
    )
    rep["cols_limpias"] = list(dfc.columns) #guarda los nombres ya limpios

 
    rep["shape"] = dfc.shape #devuelve funciones basicas de pandas. Devuelve el tamaño y el tipo de dato de cada columna.
    rep["dtypes"] = dfc.dtypes

  
    rep["nulos"] = dfc.isnull().sum().sort_values(ascending=False)#saca los nulos y los duplicados.
    rep["duplicados"] = int(dfc.duplicated().sum())
    
    # === Print ordenado ===
    print(f"\n== {name.upper()} ==") #Esto es solo para quede bonito en Mayúscula.
    print(f"shape: {rep['shape']}\n")

    print("tipos:")
    print(rep["dtypes"].to_string())#el tipo,nulos,duplicados en texto plano para que no se corte 

    print("Nulos por columna:")
    print(rep["nulos"].to_string())


    print(f"\nDuplicados totales: {rep['duplicados']}")

    return dfc #devuelve en un dataframe limpio 



In [46]:
df_all_limpio = pre_eda(df_all, name="df_all")



== DF_ALL ==
shape: (1510, 36)

tipos:
country                          object
region                           object
happiness_rank                  float64
happiness_score                 float64
standard_error                  float64
economy_gdp_per_capita          float64
family                          float64
health_life_expectancy          float64
freedom                         float64
trust_government_corruption     float64
generosity                      float64
dystopia_residual               float64
year                              int64
lower_confidence_interval       float64
upper_confidence_interval       float64
happinessrank                   float64
happinessscore                  float64
whiskerhigh                     float64
whiskerlow                      float64
economygdppercapita             float64
healthlifeexpectancy            float64
trustgovernmentcorruption       float64
dystopiaresidual                float64
overall_rank                    float64


In [47]:
def ver_nulos(df):
    nulos = df.isna().sum()
    nulos = nulos[nulos > 0].sort_values(ascending=False)
    return nulos


In [48]:
ver_nulos(df_all_limpio)


whiskerlow                      1355
whiskerhigh                     1355
happinessscore                  1355
happinessrank                   1355
dystopiaresidual                1355
trustgovernmentcorruption       1355
economygdppercapita             1355
healthlifeexpectancy            1355
upper_confidence_interval       1353
lower_confidence_interval       1353
standard_error                  1352
score                           1198
gdp_per_capita                  1198
country_or_region               1198
overall_rank                    1198
trust_government_corruption     1195
happiness_score                 1195
region                          1195
health_life_expectancy          1195
economy_gdp_per_capita          1195
dystopia_residual               1195
family                          1040
country                         1040
freedom                         1040
economy_gdp_per_capita           785
happiness_score                  782
lowerwhisker                     782
u

In [49]:
# Ver cuántas columnas tiene cada dataset
datasets = {
    2015: df_2015,
    2016: df_2016,
    2017: df_2017,
    2018: df_2018,
    2019: df_2019,
    2020: df_2020,
    2021: df_2021,
    2022: df_2022,
    2023: df_2023,
    2024: df_2024
}

for year, df in datasets.items():
    print(f"{year}: {df.shape[1]} columnas")
    print(df.columns.tolist())
    print("-" * 60)


2015: 12 columnas
['Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Standard Error', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual']
------------------------------------------------------------
2016: 13 columnas
['Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Lower Confidence Interval', 'Upper Confidence Interval', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual']
------------------------------------------------------------
2017: 12 columnas
['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high', 'Whisker.low', 'Economy..GDP.per.Capita.', 'Family', 'Health..Life.Expectancy.', 'Freedom', 'Generosity', 'Trust..Government.Corruption.', 'Dystopia.Residual']
------------------------------------------------------------
2018: 9 columnas
['Overall rank', 'Country or region',

In [50]:


# 1) Estándar de columnas que quiero
columnas_objetivo = [
    'Country', 'Happiness Rank', 'Happiness Score',
    'Upperwhisker', 'Lowerwhisker',
    'Economy (GDP per Capita)', 'Social support',
    'Healthy life expectancy', 'Freedom to make life choices',
    'Generosity', 'Perceptions of corruption', 'Year'
]

# 2) Variantes de nombres -> nombre estándar
equivalencias = {
    'Country name': 'Country',
    'Country or region': 'Country',
    'Happiness score': 'Happiness Score',
    'Happinessscore': 'Happiness Score',
    'Happiness.rank': 'Happiness Rank',
    'Overall rank': 'Happiness Rank',
    'Whisker.high': 'Upperwhisker',
    'Whisker.low': 'Lowerwhisker',
    'Economy..GDP.per.Capita.': 'Economy (GDP per Capita)',
    'Economy GDP per capita': 'Economy (GDP per Capita)',
    'Health (Life Expectancy)': 'Healthy life expectancy',
    'Trust (Government Corruption)': 'Perceptions of corruption'
}

# 3) Lista (no dict) de dataframes con su año
dfs = [
    (2015, df_2015), (2016, df_2016), (2017, df_2017), (2018, df_2018),
    (2019, df_2019), (2020, df_2020), (2021, df_2021), (2022, df_2022),
    (2023, df_2023), (2024, df_2024)
]

# 4) Normaliza, completa faltantes, ordena y concatena
normalizados = []
for year, df in dfs:
    df = df.copy()
    df.rename(columns=equivalencias, inplace=True)
    
    # Asegura nombres únicos
    df.columns = df.columns.str.strip()
    df = df.loc[:, ~df.columns.duplicated()]

    # Añade columnas faltantes
    for col in columnas_objetivo:
        if col not in df.columns:
            df[col] = None

    # Solo conserva columnas objetivo
    df = df[[col for col in columnas_objetivo if col in df.columns]]
    df["Year"] = year
    normalizados.append(df)

df_all = pd.concat(normalizados, ignore_index=True)
df_all.to_csv("world_happiness_all_years_clean.csv", index=False)
print("OK → world_happiness_all_years_clean.csv")


OK → world_happiness_all_years_clean.csv


C:\Users\elaru\AppData\Local\Temp\ipykernel_21276\2072230471.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat(normalizados, ignore_index=True)


In [51]:
def verificar_equivalencias(df, equivalencias, columnas_objetivo):
    cols = set(df.columns)
    claves = set(equivalencias.keys())
    valores = set(equivalencias.values()) | set(columnas_objetivo)

    sin_cubrir = cols - claves - valores
    print("Columnas sin equivalencia o fuera del objetivo:")
    print(sin_cubrir)


In [52]:
verificar_equivalencias(df_all, equivalencias, columnas_objetivo)

Columnas sin equivalencia o fuera del objetivo:
set()


In [53]:
pre_eda(df_all, name="df_all_final")



== DF_ALL_FINAL ==
shape: (1510, 12)

tipos:
country                          object
happiness_rank                   object
happiness_score                 float64
upperwhisker                    float64
lowerwhisker                    float64
economy_gdp_per_capita          float64
social_support                  float64
healthy_life_expectancy         float64
freedom_to_make_life_choices    float64
generosity                      float64
perceptions_of_corruption       float64
year                              int64
Nulos por columna:
upperwhisker                    627
lowerwhisker                    627
social_support                  473
freedom_to_make_life_choices    473
happiness_score                 467
economy_gdp_per_capita          315
healthy_life_expectancy         159
perceptions_of_corruption       159
happiness_rank                  155
generosity                        3
country                           0
year                              0

Duplicados totales: 0


,country,happiness_rank,happiness_score,upperwhisker,lowerwhisker,economy_gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption,year
0,Switzerland,1,7.587,NaN,NaN,1.39651,NaN,0.94143,NaN,0.29678,0.41978,2015
1,Iceland,2,7.561,NaN,NaN,1.30232,NaN,0.94784,NaN,0.43630,0.14145,2015
2,Denmark,3,7.527,NaN,NaN,1.32548,NaN,0.87464,NaN,0.34139,0.48357,2015
3,Norway,4,7.522,NaN,NaN,1.45900,NaN,0.88521,NaN,0.34699,0.36503,2015
4,Canada,5,7.427,NaN,NaN,1.32629,NaN,0.90563,NaN,0.45811,0.32957,2015
...,...,...,...,...,...,...,...,...,...,...,...,...
1505,Congo (Kinshasa),139,3.295,3.462,3.128,0.53400,0.665,0.26200,0.473,0.18900,0.07200,2024
1506,Sierra Leone,140,3.245,3.366,3.124,0.65400,0.566,0.25300,0.469,0.18100,0.05300,2024
1507,Lesotho,141,3.186,3.469,2.904,0.77100,0.851,0.00000,0.523,0.08200,0.08500,2024
1508,Lebanon,142,2.707,2.797,2.616,1.37700,0.577,0.55600,0.173,0.06800,0.02900,2024


In [54]:
def resumen_nulos(df):
    total = len(df)
    resumen = (
        df.isna().sum()
        .to_frame("nulos")
        .assign(porcentaje=lambda x: (x["nulos"] / total * 100).round(2))
        .sort_values("nulos", ascending=False)
    )
    return resumen


In [55]:
resumen_nulos(df_all)


,nulos,porcentaje
Upperwhisker,627,41.52
Lowerwhisker,627,41.52
Social support,473,31.32
Freedom to make life choices,473,31.32
Happiness Score,467,30.93
Economy (GDP per Capita),315,20.86
Healthy life expectancy,159,10.53
Perceptions of corruption,159,10.53
Happiness Rank,155,10.26
Generosity,3,0.20


In [56]:
df_all = df_all.drop(columns=['Upperwhisker', 'Lowerwhisker'])


In [57]:
cols_imputar = [
    'Social support',
    'Freedom to make life choices',
    'Happiness Score',
    'Economy (GDP per Capita)',
    'Healthy life expectancy',
    'Perceptions of corruption'
]

for col in cols_imputar:
    df_all[col].fillna(df_all[col].mean(), inplace=True)


C:\Users\elaru\AppData\Local\Temp\ipykernel_21276\2894680039.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_all[col].fillna(df_all[col].mean(), inplace=True)


In [60]:
# Asegura numéricos
df_all['Happiness Score'] = pd.to_numeric(df_all['Happiness Score'], errors='coerce')
df_all['Year'] = pd.to_numeric(df_all['Year'], errors='coerce')

# Recalcula ranking por año (sin NaN)
df_all['Happiness Rank'] = (
    df_all.groupby('Year')['Happiness Score']
          .rank(ascending=False, method='dense')
)


In [61]:
df_all['Generosity'] = pd.to_numeric(df_all['Generosity'], errors='coerce')
gen_country_mean = df_all.groupby('Country')['Generosity'].transform('mean')
df_all['Generosity'] = df_all['Generosity'].fillna(gen_country_mean)
df_all['Generosity'] = df_all['Generosity'].fillna(df_all['Generosity'].mean())


In [62]:
print(df_all.isna().sum())
df_all.to_csv('world_happiness_all_years_ready.csv', index=False)


Country                         0
Happiness Rank                  0
Happiness Score                 0
Economy (GDP per Capita)        0
Social support                  0
Healthy life expectancy         0
Freedom to make life choices    0
Generosity                      0
Perceptions of corruption       0
Year                            0
dtype: int64


In [64]:
df_all.columns


Index(['Country', 'Happiness Rank', 'Happiness Score',
       'Economy (GDP per Capita)', 'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Year'],
      dtype='object')

In [66]:
import os
os.listdir()


['2015.csv',
 '2016.csv',
 '2017.csv',
 '2018.csv',
 '2019.csv',
 '2020.csv',
 '2021.csv',
 '2022.csv',
 '2023.csv',
 '2024.csv',
 'eda_para_todos_csv.ipynb',
 'libro1Proyecto_definitivo.twb',
 'world_happiness_all_years.csv',
 'world_happiness_all_years_clean.csv',
 'world_happiness_all_years_ready.csv',
 '~libro1Proyecto_definitivo__14228.twbr',
 '~libro1Proyecto_definitivo__21508.twbr']